<a href="https://colab.research.google.com/github/frost-nasib/Supply-chain-Project/blob/main/Arman_Rashid_Project_(Task_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Load the data
file_path = 'arman.csv'
try:
    data = pd.read_csv(file_path, on_bad_lines='skip')
except pd.errors.ParserError as e:
    print(f"Error reading CSV file: {e}")
    exit()

# Define necessary columns
required_columns = ['Product ID', 'Product Name', 'Current Stock', 'Reorder Level', 'Maximum Stock', 'Supplier ID', 'Sales']

# Ensure the necessary columns exist
for col in required_columns:
    if col not in data.columns:
        print(f"Warning: Missing required column: {col}. Filling with default values.")
        if col == 'Sales':
            data[col] = 0  # Default value for missing Sales column
        else:
            data[col] = np.nan

# Inventory Management
def track_inventory(df):
    df['Reorder Alert'] = df['Current Stock'] < df['Reorder Level']
    df['Overstock Alert'] = df['Current Stock'] > df['Maximum Stock']
    return df[['Product ID', 'Product Name', 'Current Stock', 'Reorder Alert', 'Overstock Alert']]

inventory_status = track_inventory(data)
print("Inventory Status:")
print(inventory_status)

# Demand Forecasting
def forecast_demand(df, periods=3):
    forecast_results = []
    for product_id in df['Product ID'].unique():
        product_data = df[df['Product ID'] == product_id]['Sales'].dropna().values
        if len(product_data) >= 12:  # Ensure enough data
            model = ExponentialSmoothing(product_data, trend='add', seasonal='add', seasonal_periods=12)
            fit_model = model.fit()
            forecast = fit_model.forecast(periods)
            forecast_results.append({'Product ID': product_id, 'Forecast': forecast.tolist()})
    return forecast_results

demand_forecast = forecast_demand(data)
print("\nDemand Forecast:")
print(demand_forecast)

# Supply Optimization
def suggest_reorder(df, forecast):
    suggestions = []
    for f in forecast:
        product = df[df['Product ID'] == f['Product ID']].iloc[0]
        suggested_qty = max(0, sum(f['Forecast']) - product['Current Stock'])
        suggestions.append({'Product ID': product['Product ID'], 'Suggested Reorder Quantity': suggested_qty})
    return suggestions

reorder_suggestions = suggest_reorder(data, demand_forecast)
print("\nReorder Suggestions:")
print(reorder_suggestions)

# Save the results to CSV
inventory_status.to_csv('inventory_status.csv', index=False)
pd.DataFrame(demand_forecast).to_csv('demand_forecast.csv', index=False)
pd.DataFrame(reorder_suggestions).to_csv('reorder_suggestions.csv', index=False)


Inventory Status:
                                                               Product ID  \
P001 Laptop         S001 15  10  50   20  25  18  22  30  28           24   
P002 Smartphone     S002 50  30  100  60  55  52  58  65  70           68   
P003 Steel Rod      S003 100 50  300  80  85  90  95  100 110         120   
P004 Toilet Paper   S004 200 100 500  150 170 160 180 200 210         220   
P005 Milk           S005 80  50  200  60  65  70  75  85  95          100   
P006 Chair          S006 10  5   30   15  12  10  14  16  18           20   
P008 Car Battery    S008 5   2   20   8   7   10  9   12  11           14   
P009 Apple          S001 120 80  300  90  85  95  100 110 115         120   
P010 Desktop        S002 20  10  60   18  22  20  25  28  32           35   
P011 Cement         S003 150 100 400  120 130 140 150 160 170         180   
P012 Soap           S004 350 200 700  250 270 260 280 300 320         340   
P013 Cheese         S005 70  40  150  60  65  70  75  80  